In [ ]:
# !pip install pyttsx3
# !pip install -U speechRecognition
# # !pip install webbrowser
# !pip install pywhatkit
# !pip install os
# !pip install yfinance
# !pip install pyjokes
# !pip install wikipedia
# !pip install flask


import pyttsx3                      
import speech_recognition as SR     
import webbrowser                   
import datetime                    
import pywhatkit                    
import os                          
import yfinance as yf               
import pyjokes                      
import wikipedia

from bs4 import BeautifulSoup
import requests
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}


#""" listen to our microphone and return the audio as text using google"""

def speech_to_text():
    """Transforms what we say to text"""
    r = SR.Recognizer()                     
    with SR.Microphone() as source:         
        r.pause_threshold = 0.8             
        said = r.listen(source)             

        try:
            print('I am listening...')
            q = r.recognize_google(said, language="en")
            return q
        except SR.UnknownValueError:
            print("Sorry, I didn't understand. Please say again.")
            return "I am waiting"
        except SR.RequestError:
            print("Sorry the service is down.")
            return "I am waiting."
        except:
            "I am waiting"



def speak(message):
    """converts text to speech"""
    engine = pyttsx3.init()
    engine.say(message)
    engine.runAndWait()


# #changing the voice if needed. (replace the id)
# engine = pyttsx3.init()
# id = 'HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0'
# engine.setProperty('voice',id)              #setting property of voice with that id
# engine.runAndWait()


#returns the weekday
def return_day():
    day = datetime.date.today()
    weekday = day.weekday()

    mapping = {
        0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6:'Sunday'
    }
    print(mapping[weekday])
    try:
        speak(f'Today is {mapping[weekday]}')
    except:
        pass



def return_time():
    time = datetime.datetime.now().strftime("%I:%M:%S")
    print(time)
    time_to_speak = f"{time[0:2]} o' clock {time[3:5]} minutes and {time[6:8]} seconds"
    speak(time_to_speak)       



def startup_script():
    """Intro greeting at startup"""
    speak('''Hi I am your personal assistant.
    You can call me David.
    How may I help you?
    ''')

def weather(city):
    city=city.replace(" ","+")
    res = requests.get(f'https://www.google.com/search?q={city}&oq={city}&aqs=chrome.0.35i39l2j0l4j46j69i60.6128j1j7&sourceid=chrome&ie=UTF-8',headers=headers)
    print("Searching in google.....")
    soup = BeautifulSoup(res.text,'html.parser')
    location = soup.select('#wob_loc')[0].getText().strip()
    time = soup.select('#wob_dts')[0].getText().strip()
    info = soup.select('#wob_dc')[0].getText().strip()
    weather = soup.select('#wob_tm')[0].getText().strip()
    
    return f"It is {weather}°C {info} in {location} at {time}"



def querying():
    startup_script()

    start = True
    while start:
        q = speech_to_text().lower()

        if 'start youtube' in q:
            speak('Opening youtube. Just a second')
            webbrowser.open('https://youtube.com')
            continue

        elif 'start browser' in q:
            speak('Opening Browser')
            webbrowser.open('https://www.google.com')
            continue

        elif 'what day is it' in q:
            return_day()
            continue

        elif 'what time is it' in q:
            return_time()
            continue

        elif 'wikipedia' in q:
            speak('Checking wikipedia')
            q = q.replace("wikipedia","")
            result = wikipedia.summary(q, sentences=2)
            speak('found on wikipedia')
            speak(result)
            continue

        elif 'your name' in q:
            speak('I am David. Your voice assistant')
            continue

        elif "search web" in q:
            pywhatkit.search(q)
            speak('That is what I found')
            continue

        elif "play" in q:
            speak(f'Playing {q}')
            pywhatkit.playonyt(q)
            continue

        elif "joke" in q:
            speak(pyjokes.get_joke())
            continue

        elif "stock price" in q:
            search = q.split("of")[-1].strip()   

            lookup = {'apple': 'AAPL', 'amazon': 'AMZN', 'google':'GOOG'}

            try:
                stock = lookup[search]
                stock = yf.Ticker(stock)
                currentprice = stock.info["regularMarketPrice"]            
                speak(f'found it, the price for {search} is {currentprice}')
                continue
            except:
                speak(f'sorry I have no data for {search}')

            continue
            
            
        elif "weather" in q:
            city = q.split("in")[-1].strip()   
            city=city+" weather"
            
            try:
                weather_info = weather(city)
                speak(weather_info)
                
                continue
            except:
                speak(f'sorry I have no data for weather in {city}')

            continue

        elif 'shutdown' in q:
            speak('Goodbye have a nice day')
            break

querying()